## Green Trips

In [1]:
import pandas as pd
import numpy as np

In [2]:
green = pd.read_csv('green_trips.csv.gz')
geography = pd.read_csv("geographic.csv")

In [3]:
green.head()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.8
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.8
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.3
3,2014-04-27 02:27:04,2014-04-27 02:39:02,-73.949501,40.713997,-73.987785,40.718582,2,3.68,17.3
4,2014-05-26 18:32:19,2014-05-26 18:44:13,-73.944092,40.672195,-73.977325,40.664013,1,2.40,11.5


In [4]:
green.shape

(3589048, 9)

In [5]:
geography.head(5)

,BK88,QN52,QN48,QN51,QN27,BX35,BX98,QN07,MN06,QN02,...,MN25,MN24,MN23,MN13,MN15,MN32,MN33,MN99,QN18,QN29
0,-73.976051,-73.794934,-73.775740,-73.803792,-73.861099,-73.896967,-73.872873,-73.757258,-73.946080,-73.760316,...,-74.000784,-73.992605,-73.989904,-74.001535,-73.993833,-73.938048,-73.935054,-74.010930,-73.858726,-73.867911
1,40.631284,40.757806,40.743334,40.775618,40.763673,40.834350,40.785983,40.718147,40.821271,40.675118,...,40.694295,40.724145,40.734443,40.762653,40.772940,40.780838,40.791695,40.684500,40.735717,40.744942
2,-73.977167,-73.794749,-73.775799,-73.800991,-73.859931,-73.896787,-73.872886,-73.755890,-73.946404,-73.758806,...,-74.000960,-73.992634,-73.989870,-74.001339,-73.993789,-73.937903,-73.935108,-74.011934,-73.858672,-73.867877
3,40.630755,40.755636,40.743326,40.775397,40.762209,40.834133,40.785971,40.716307,40.820580,40.673014,...,40.694077,40.724065,40.733535,40.762217,40.772912,40.780758,40.791658,40.683896,40.735627,40.744824
4,-73.977000,-73.794579,-73.778060,-73.798653,-73.859668,-73.896303,-73.872983,-73.754702,-73.946720,-73.758598,...,-74.003015,-73.993093,-73.990028,-74.001279,-73.993727,-73.937796,-73.935173,-74.012177,-73.858585,-73.867852


In [6]:
# Revisar si existen valores nulos (NaNs)
# green.isnull().sum()

El siguiente procedimiento convierte el formato leído a un formato similar al .wkt que lee la librería de gráficos `geopandas`.

In [7]:
geodf_raw = geography.transpose()

In [8]:
ntas_names = list(geodf_raw.index)
polygons = []
for i in range(len(geodf_raw)):
    string = 'POLYGON (('
    coord = ''
    for j in geodf_raw.columns:
        if (not pd.isna(geodf_raw.iloc[i,j])): 
            if j % 2 == 0:
                coord = str(geodf_raw.iloc[i,j])
            elif j % 2 == 1:
                coord = coord + ' ' + str(geodf_raw.iloc[i,j]) + ','
                string = string + coord
    string = string + str(geodf_raw.iloc[i,0]) + ' ' + str(geodf_raw.iloc[i,1]) + '))'
    polygons.append(string)
    
ntas_df = pd.DataFrame( {'name':ntas_names, 'geometry':polygons} )

In [9]:
ntas_df.head()

,name,geometry
0,BK88,"POLYGON ((-73.9760507905698 40.6312841471042,-..."
1,QN52,"POLYGON ((-73.7949338459459 40.7578063026709,-..."
2,QN48,POLYGON ((-73.7757397458527 40.743333917592295...
3,QN51,"POLYGON ((-73.8037916164017 40.7756183875692,-..."
4,QN27,"POLYGON ((-73.8610986485262 40.7636727485249,-..."


In [10]:
#ntas_df.to_csv('ntas_df.csv')

In [11]:
import geopandas
from shapely import wkt

In [12]:
ntas_df['geometry'] = ntas_df['geometry'].apply(wkt.loads)

In [13]:
gdf = geopandas.GeoDataFrame(ntas_df, geometry='geometry')
print(gdf.head())

   name                                           geometry
0  BK88  POLYGON ((-73.9760507905698 40.6312841471042, ...
1  QN52  POLYGON ((-73.79493384594591 40.7578063026709,...
2  QN48  POLYGON ((-73.7757397458527 40.7433339175923, ...
3  QN51  POLYGON ((-73.8037916164017 40.7756183875692, ...
4  QN27  POLYGON ((-73.86109864852619 40.7636727485249,...


In [14]:
gdf.plot();

### Leyendo green_trips y convirtiendo puntos en NTAs

In [15]:
from shapely.geometry import Point, Polygon

In [16]:
point_list = []
for i in list(zip(list(green['pickup_latitude']),list(green['pickup_longitude']))):
    point_list.append(Point(i[1],i[0]))
    
point_list_df = pd.DataFrame(columns = ['geometry'], data = point_list)

In [17]:
point_list_df.head()

,geometry
0,POINT (-73.9535446167 40.8111724854)
1,POINT (-73.9467086792 40.7143211365)
2,POINT (-73.9576263428 40.7180938721)
3,POINT (-73.9495010376 40.7139968872)
4,POINT (-73.9440917969 40.6721954346)


In [18]:
points_gdf = geopandas.GeoDataFrame(point_list_df, geometry='geometry')
print(points_gdf.head())

                               geometry
0  POINT (-73.9535446167 40.8111724854)
1  POINT (-73.9467086792 40.7143211365)
2  POINT (-73.9576263428 40.7180938721)
3  POINT (-73.9495010376 40.7139968872)
4  POINT (-73.9440917969 40.6721954346)


In [19]:
points = points_gdf
polys = gdf
pointInPoly = geopandas.sjoin(points, polys, op='within')

In [20]:
pointInPoly.head()

,geometry,index_right,name
0,POINT (-73.9535446167 40.8111724854),148,MN09
39,POINT (-73.96208190919999 40.8052177429),148,MN09
41,POINT (-73.96212768550001 40.8054428101),148,MN09
53,POINT (-73.9570465088 40.8125801086),148,MN09
60,POINT (-73.95924377439999 40.8092575073),148,MN09


In [21]:
green_nta = green.join(pointInPoly['name'], how='left')
green_nta = green_nta.rename(columns={'name': 'nta_code'})

In [22]:
green_nta.head()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,nta_code
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.8,MN09
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.8,BK90
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.3,BK73
3,2014-04-27 02:27:04,2014-04-27 02:39:02,-73.949501,40.713997,-73.987785,40.718582,2,3.68,17.3,BK73
4,2014-05-26 18:32:19,2014-05-26 18:44:13,-73.944092,40.672195,-73.977325,40.664013,1,2.40,11.5,BK61


In [23]:
green_nta.shape

(3589048, 10)

In [24]:
# Valores nulos (NaNs) en green_nta
green_nta.isnull().sum()

pickup_datetime          0
dropoff_datetime         0
pickup_longitude         0
pickup_latitude          0
dropoff_longitude        0
dropoff_latitude         0
passenger_count          0
trip_distance            0
total_amount             0
nta_code             97549
dtype: int64

In [25]:
# Filas con valores NaN en cualquier columna
# green_nta[green_nta.isnull().any(axis=1)]

### Merge con Zonas

In [26]:
zones = pd.read_csv("zones.csv")

In [68]:
# Encontrar nta_code duplicados
names = zones.nta_code.value_counts()
print(len(names[names > 1]))
# Ejemplo de filas duplicadas por nta-code
# Se puede eliminar la columna zone?
zones.loc[zones['nta_code']=='QN70']

44


,location_id,borough,zone,service_zone,nta_code
6,7,Queens,Astoria,Boro Zone,QN70
7,8,Queens,Astoria Park,Boro Zone,QN70


In [27]:
# Eliminar filas de 'zones' con valores duplicados en 'nta_code'.
# Para evitar filas duplicadas luego del merge.
zones_clean = zones.drop_duplicates(subset=['nta_code'])

In [56]:
zones_clean.head(10)

,location_id,borough,zone,service_zone,nta_code
0,1,EWR,Newark Airport,EWR,NJ01
1,2,Queens,Jamaica Bay,Boro Zone,QN61
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,BX31
3,4,Manhattan,Alphabet City,Yellow Zone,MN22
4,5,Staten Island,Arden Heights,Boro Zone,SI48
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone,SI14
6,7,Queens,Astoria,Boro Zone,QN70
8,9,Queens,Auburndale,Boro Zone,QN48
9,10,Queens,Baisley Park,Boro Zone,QN76
10,11,Brooklyn,Bath Beach,Boro Zone,BK27


In [29]:
# green_zones = green_nta.join(zones, on='nta_code')
green_zones = pd.merge(green_nta, zones_clean, on='nta_code', how='left')
# green_zones = green_zones.rename(columns={'name': 'nta_code'})

In [70]:
green_zones.head(100)

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,nta_code,location_id,borough,zone,service_zone
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.80,MN09,166.0,Manhattan,Morningside Heights,Boro Zone
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.80,BK90,80.0,Brooklyn,East Williamsburg,Boro Zone
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.30,BK73,255.0,Brooklyn,Williamsburg (North Side),Boro Zone
3,2014-04-27 02:27:04,2014-04-27 02:39:02,-73.949501,40.713997,-73.987785,40.718582,2,3.68,17.30,BK73,255.0,Brooklyn,Williamsburg (North Side),Boro Zone
4,2014-05-26 18:32:19,2014-05-26 18:44:13,-73.944092,40.672195,-73.977325,40.664013,1,2.40,11.50,BK61,61.0,Brooklyn,Crown Heights North,Boro Zone
5,2015-03-04 21:36:48,2015-03-04 21:43:47,-73.954826,40.804962,-73.944008,40.812088,1,1.16,9.36,MN11,NaN,NaN,NaN,NaN
6,2015-01-21 09:27:41,2015-01-21 09:51:01,-73.862000,40.730438,-73.865089,40.656422,1,7.50,25.80,QN18,196.0,Queens,Rego Park,Boro Zone
7,2015-03-07 18:51:58,2015-03-07 19:20:49,-73.971947,40.675697,-74.009293,40.635708,6,4.58,21.30,BK37,106.0,Brooklyn,Gowanus,Boro Zone
8,2015-01-11 16:55:04,2015-01-11 17:04:26,-73.949883,40.681896,-73.923279,40.684875,1,1.51,8.80,BK75,17.0,Brooklyn,Bedford,Boro Zone
9,2014-05-30 05:53:15,2014-05-30 06:00:00,-73.952370,40.789875,-73.949173,40.781448,1,1.20,7.50,MN33,75.0,Manhattan,East Harlem South,Boro Zone


In [31]:
# NTAs que están en GEOGRAPHIC, pero no en ZONES
aux = green_zones[green_zones['nta_code'].notnull()]
aux = aux[aux.isnull().any(axis=1)]
aux.groupby(['nta_code'])['total_amount'].count()

nta_code
BK23      1075
BX99        85
MN11    171143
QN99         6
SI05        22
Name: total_amount, dtype: int64

In [32]:
# Parámetros con valores en nulo luego de hacer el merge entre green_nta y zones_clean
green_zones.isnull().sum()

pickup_datetime           0
dropoff_datetime          0
pickup_longitude          0
pickup_latitude           0
dropoff_longitude         0
dropoff_latitude          0
passenger_count           0
trip_distance             0
total_amount              0
nta_code              97549
location_id          269880
borough              269880
zone                 269880
service_zone         269880
dtype: int64

### Merge con Demographics

In [43]:
demographics = pd.read_csv("demographics.csv")

In [47]:
demographics.columns

Index(['nta_name', 'borough', 'nta_code', 'population', 'under_5_years',
       '5-9_years', '10-14_years', '15-19_years', '20-24_years', '25-29_years',
       '30-34_years', '35-39_years', '40-44_years', '45-49_years',
       '50-54_years', '55-59_years', '60-64_years', 'over_65_years',
       'median_age', 'people_per_acre', 'households', 'less_than_10,000',
       '10000_to_14999', '15000_to_24999', '25000_to_34999', '35000_to_49999',
       '50000_to_74999', '75000_to_99999', '100000_to_149999',
       '150000_to_199999', '200000_or_more', 'median_income', 'mean_income'],
      dtype='object')

In [46]:
# demographics.head()

,nta_name,borough,nta_code,population,under_5_years,5-9_years,10-14_years,15-19_years,20-24_years,25-29_years,...,15000_to_24999,25000_to_34999,35000_to_49999,50000_to_74999,75000_to_99999,100000_to_149999,150000_to_199999,200000_or_more,median_income,mean_income
0,Allerton-Pelham Gardens,Bronx,BX31,28903,1679,1706,1763,2039,1964,1703,...,797,773,1160,1764,1155,1562,765,427,61638,78489
1,Annadale-Huguenot-Prince's Bay-Eltingville,Staten Island,SI01,27770,1397,1698,1817,1880,1720,1594,...,571,405,1008,1523,1346,2075,1086,1151,88288,109187
2,Arden Heights,Staten Island,SI48,25238,1507,1540,1596,1752,1614,1561,...,337,516,707,1421,1611,2021,1047,740,89570,101627
3,Astoria,Queens,QN70,78793,3480,3037,3060,3392,6630,11586,...,3673,2816,4725,6463,4557,4698,1627,1197,54882,70094
4,Auburndale,Queens,QN48,19996,917,966,1063,1168,1214,1307,...,445,632,690,1417,1060,1237,589,433,70772,84402


In [69]:
# Hay filas duplicadas? No
#demographics.loc[demographics['nta_code']=='QN48']
names = demographics.nta_code.value_counts()
len(names[names > 1])

0

In [63]:
green_demo = pd.merge(green_zones, demographics, on='nta_code', how='left')

In [65]:
green_demo.columns

Index(['pickup_datetime', 'dropoff_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'trip_distance', 'total_amount', 'nta_code',
       'location_id', 'borough_x', 'zone', 'service_zone', 'nta_name',
       'borough_y', 'population', 'under_5_years', '5-9_years', '10-14_years',
       '15-19_years', '20-24_years', '25-29_years', '30-34_years',
       '35-39_years', '40-44_years', '45-49_years', '50-54_years',
       '55-59_years', '60-64_years', 'over_65_years', 'median_age',
       'people_per_acre', 'households', 'less_than_10,000', '10000_to_14999',
       '15000_to_24999', '25000_to_34999', '35000_to_49999', '50000_to_74999',
       '75000_to_99999', '100000_to_149999', '150000_to_199999',
       '200000_or_more', 'median_income', 'mean_income'],
      dtype='object')

In [64]:
green_demo.head()

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,total_amount,nta_code,...,15000_to_24999,25000_to_34999,35000_to_49999,50000_to_74999,75000_to_99999,100000_to_149999,150000_to_199999,200000_or_more,median_income,mean_income
0,2015-02-01 01:26:45,2015-02-01 01:49:58,-73.953545,40.811172,-73.984764,40.728386,1,8.11,27.8,MN09,...,1979.0,1784.0,1482.0,2381.0,2009.0,2193.0,1385.0,2514.0,56371.0,96085.0
1,2015-01-02 20:06:28,2015-01-02 20:14:04,-73.946709,40.714321,-73.961571,40.711475,1,1.29,9.8,BK90,...,1902.0,1426.0,1610.0,2243.0,1693.0,2219.0,1002.0,618.0,50777.0,69329.0
2,2014-09-27 17:55:38,2014-09-27 18:19:56,-73.957626,40.718094,-73.947304,40.777813,5,6.12,26.3,BK73,...,1831.0,2071.0,2038.0,3158.0,2292.0,2934.0,1642.0,2051.0,61381.0,89513.0
3,2014-04-27 02:27:04,2014-04-27 02:39:02,-73.949501,40.713997,-73.987785,40.718582,2,3.68,17.3,BK73,...,1831.0,2071.0,2038.0,3158.0,2292.0,2934.0,1642.0,2051.0,61381.0,89513.0
4,2014-05-26 18:32:19,2014-05-26 18:44:13,-73.944092,40.672195,-73.977325,40.664013,1,2.40,11.5,BK61,...,5231.0,4543.0,5901.0,6692.0,3927.0,3466.0,1089.0,686.0,37104.0,51519.0


### Merge con Weather

In [33]:
weather = pd.read_csv("weather.csv")

In [34]:
weather.head()

,date,max_temp,min_temp,avg_temp,precipitation,snowfall,snow_depth,location,latitude,longitude
0,1/1/14,34,24,29.0,0,0,0,JFK,40.64749,-73.779982
1,1/2/14,33,19,26.0,0.13,2.4,0,JFK,40.64749,-73.779982
2,1/3/14,19,9,14.0,0.22,5.5,7,JFK,40.64749,-73.779982
3,1/4/14,31,3,17.0,0,0,7,JFK,40.64749,-73.779982
4,1/5/14,49,23,36.0,0.1,,6,JFK,40.64749,-73.779982


In [38]:
weather['date'] = pd.to_datetime(weather['date'])

In [42]:
weather.head()

,date,max_temp,min_temp,avg_temp,precipitation,snowfall,snow_depth,location,latitude,longitude
0,2014-01-01,34,24,29.0,0,0,0,JFK,40.64749,-73.779982
1,2014-01-02,33,19,26.0,0.13,2.4,0,JFK,40.64749,-73.779982
2,2014-01-03,19,9,14.0,0.22,5.5,7,JFK,40.64749,-73.779982
3,2014-01-04,31,3,17.0,0,0,7,JFK,40.64749,-73.779982
4,2014-01-05,49,23,36.0,0.1,,6,JFK,40.64749,-73.779982


#### Total de pickup por cada NTA

In [35]:
# Total de pickup por cada NTA
df_count_group_by_NTA = pointInPoly.groupby(pointInPoly['name']).size()
print("Total NTAs: " + str(df_count_group_by_NTA.sort_values(ascending=[False]).shape[0]))
df_count_group_by_NTA.sort_values(ascending=[False]).head(10)

Total NTAs: 178


name
BK73    262645
QN70    173194
MN34    171974
BK38    171813
MN11    171143
MN33    153241
MN09    150311
QN29    136821
BK37    123653
MN03    121056
dtype: int64